## this gets the data into csv format

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#this is commented out because it takes forever and I will just provide the filtered files. 
#however it is good to see how we did this

#open('filtered_data\q1_commit_messages_filtered.txt','w',encoding="utf8").writelines(line.replace('][',' ||| ').replace('] = ', ' ||| ').replace('commit_log[','') for line in open('raw_data\q1_commit_messages.txt',encoding="utf8") if 'commit_log' in line)
#open('filtered_data\q2_commit_dates_epoch_timestamp_filtered.txt','w',encoding="utf8").writelines(line.replace('][',' ||| ').replace('] = ', ' ||| ').replace('commit_log[','') for line in open('raw_data\q2_commit_dates_epoch_timestamp.txt',encoding="utf8") if 'commit_log' in line)
#open('filtered_data\q3_author_names_filtered.txt','w',encoding="utf8").writelines(line.replace('][',' ||| ').replace('] = ', ' ||| ').replace('commit_log[','') for line in open('raw_data\q3_author_names.txt',encoding="utf8") if 'commit_log' in line)
#open('filtered_data\q4_committer_names_filtered.txt','w',encoding="utf8").writelines(line.replace('][',' ||| ').replace('] = ', ' ||| ').replace('commit_log[','') for line in open('raw_data\q4_committer_names.txt',encoding="utf8") if 'commit_log' in line)
#open('filtered_data\q5_all_changed_files_filtered.txt','w',encoding="utf8").writelines(line.replace('][',' ||| ').replace('] = ', ' ||| ').replace('commit_log[','') for line in open('raw_data\q5_all_changed_files.txt',encoding="utf8") if 'commit_log' in line)
#open('filtered_data\q6_deleted_files_filtered.txt','w',encoding="utf8").writelines(line.replace('][',' ||| ').replace('] = ', ' ||| ').replace('commit_log[','') for line in open('raw_data\q6_deleted_files.txt',encoding="utf8") if 'commit_log' in line)
#open('filtered_data\q7_added_files_filtered.txt','w',encoding="utf8").writelines(line.replace('][',' ||| ').replace('] = ', ' ||| ').replace('commit_log[','') for line in open('raw_data\q7_added_files.txt',encoding="utf8") if 'commit_log' in line)
#open('filtered_data\q8_modified_files_filtered.txt','w',encoding="utf8").writelines(line.replace('][',' ||| ').replace('] = ', ' ||| ').replace('commit_log[','') for line in open('raw_data\q8_modified_files.txt',encoding="utf8") if 'commit_log' in line)
#open('filtered_data\q10_commit_count_by_project_filtered.txt','w',encoding="utf8").writelines(line.replace('][',' ||| ').replace('] = ', ' ||| ').replace('commit_log[','') for line in open('raw_data\q10_commit_count_by_project.txt',encoding="utf8") if 'commit_log' in line)

## here we bring the filtered files in and merge the first set into one dataframe

In [ ]:
messages = pd.read_csv('filtered_data/q1_commit_messages_filtered.txt', usecols=[0,1,2], sep='\|\|\|', names=['project_id','commit_id','message'])
timestamps = pd.read_csv('filtered_data/q2_commit_dates_epoch_timestamp_filtered.txt', usecols=[0,1,2], sep='\|\|\|', names=['project_id','commit_id','timestamp'])
authors = pd.read_csv('filtered_data/q3_author_names_filtered.txt', usecols=[0,1,2], sep='\|\|\|', names=['project_id','author_name','commit_id'])
committers = pd.read_csv('filtered_data/q4_committer_names_filtered.txt', usecols=[0,1,2], sep='\|\|\|', names=['project_id','commit_id','committer_name'])

In [ ]:
#merge all the message related files together
merge1 = pd.merge(messages,timestamps, on=['project_id','commit_id'])
merge2 = pd.merge(merge1, authors, on=['project_id','commit_id'])
merged_data = pd.merge(merge2,committers, on=['project_id','commit_id'])

In [ ]:
#drop any rows that dont have a message because we dont need them
merged_data.dropna(subset=['message'], inplace=True)

In [ ]:
#spit out all the messages to a file alone for processing by sentistrength
merged_data.message.to_csv('sentistrength_run/messages_only.csv', header=False, index=False)

## here, we manually run the sentistrength tool on the messages_only file, which produces the messages_only_results file which we can then merge with the main dataframe

In [ ]:
#bring the strengths data in from the sentistrength results file
strengths = pd.read_csv('sentistrength_run/messages_only_results.txt', usecols=[1,2], sep='\t', dtype=str, names=['positive_strength','negative_strength'])

In [ ]:
#merge the strengths with the main dataframe
merged_data.reset_index(drop=True, inplace=True)
commit_messages_final = pd.concat([merged_data,strengths],axis=1)

In [ ]:
#convert the timestamp to date and add a day of week field
commit_messages_final["timestamp"] = commit_messages_final["timestamp"].astype(str)
commit_messages_final["timestamp"] = commit_messages_final["timestamp"].str[:10]
commit_messages_final['timestamp'] = pd.to_datetime(commit_messages_final['timestamp'],unit='s',errors='coerce')
commit_messages_final['day_of_week'] = commit_messages_final['timestamp'].dt.weekday_name
commit_messages_final['year'] = commit_messages_final['timestamp'].dt.year

In [ ]:
#compute final sentiment value from sum of positive and negative
commit_messages_final['final_sentiment'] = pd.to_numeric(commit_messages_final['positive_strength']) + pd.to_numeric(commit_messages_final['negative_strength'])

## done with commit messages, moving on to changed files

In [ ]:
#Bring in all the file related stuff
all_changes = pd.read_csv('filtered_data/q5_all_changed_files_filtered.txt', usecols=[0,1], sep='\|\|\|', names=['commit_id','filepath'])
deletes = pd.read_csv('filtered_data/q6_deleted_files_filtered.txt', usecols=[0,1], sep='\|\|\|', names=['commit_id','filepath'])
adds = pd.read_csv('filtered_data/q7_added_files_filtered.txt', usecols=[0,1], sep='\|\|\|', names=['commit_id','filepath'])
modifies = pd.read_csv('filtered_data/q8_modified_files_filtered.txt', usecols=[0,1], sep='\|\|\|', names=['commit_id','filepath'])

In [ ]:
#tag the deletes
merge_deletes = pd.merge(all_changes, deletes, on=['commit_id','filepath'], how='left', indicator='modif')
merge_deletes['modif'] = np.where(merge_deletes.modif == 'both', 'Delete', '')

In [ ]:
#tag the adds
merge_adds = pd.merge(all_changes, adds, on=['commit_id','filepath'], how='left', indicator='modif')
merge_adds['modif'] = np.where(merge_adds.modif == 'both', 'Add', '')

In [ ]:
#tag the modifies
merge_modifies = pd.merge(all_changes, modifies, on=['commit_id','filepath'], how='left', indicator='modif')
merge_modifies['modif'] = np.where(merge_modifies.modif == 'both', 'Modify', '')

In [ ]:
#merge the deletes and adds tag frames together
temp_files_merge = pd.merge(merge_deletes, merge_adds, on=['commit_id','filepath'])

In [ ]:
#squash down the columns in the delete/add frame so the tags are in the same column
temp_files_merge['modif'] = temp_files_merge['modif_x'].where(temp_files_merge['modif_x'] != '', temp_files_merge['modif_y'])
temp_files_merge = temp_files_merge.drop(['modif_x','modif_y'],axis=1)

In [ ]:
#merge the delete/add frame and the modify frame together
temp_files_merge2 = pd.merge(temp_files_merge, merge_modifies, on=['commit_id','filepath'])

In [ ]:
#squash down the columns in the final frame so the all tags are in the same column
temp_files_merge2['modification'] = temp_files_merge2['modif_x'].where(temp_files_merge2['modif_x'] != '', temp_files_merge2['modif_y'])
files_changed_final = temp_files_merge2.drop(['modif_x','modif_y'],axis=1)

##  here are the final pandas dataframes:

In [ ]:
commit_messages_final.head()

In [ ]:
files_changed_final.head()

# Analysis

### RQ1

In [ ]:
#count the number of commits for each sentiment level
commits_by_sentiment = commit_messages_final.groupby(['final_sentiment']).count().reset_index()
commits_by_sentiment = commits_by_sentiment[['final_sentiment','project_id']]
commits_by_sentiment.rename(columns={'project_id':'count'}, inplace=True)

In [ ]:
#bin the commits by positive, negative, and neutral sentiment
commits_by_sentiment['percent_of_total'] = commits_by_sentiment['count'].divide(len(commit_messages_final)).multiply(100)
commits_by_sentiment['bin'] = pd.cut(commits_by_sentiment['final_sentiment'], [-5, -1, 0, 4], labels=['negative', 'neutral', 'positive'])
commits_by_sentiment

In [ ]:
#sum up the total percent of each bin
commits_by_sentiment_binned = commits_by_sentiment.groupby(['bin']).sum().reset_index()
commits_by_sentiment_binned = commits_by_sentiment_binned[['bin','count','percent_of_total']]
commits_by_sentiment_binned

In [ ]:
commits_by_project = commit_messages_final.groupby(['project_id']).count().reset_index()

In [ ]:
commits_by_project = commits_by_project[['project_id','commit_id']]
commits_by_project.rename(columns={'commit_id':'count'}, inplace=True)
commits_by_project.sort_values(['count'], inplace=True)

In [ ]:
commits_by_project.to_csv('commits_by_project_sorted.csv', sep='\t', encoding='utf-8')

### manually obtained large, average and low groups by examining file

In [ ]:
largelist = ['12496978','10613094','5153143','7785050','1968812']
avglist = ['2424377','5256179','365893','6719407','13010741']
lowlist = ['10453653','10530838','11416657','4067771','1571039']

In [ ]:
large_projects = commit_messages_final.loc[commit_messages_final['project_id'].isin(largelist)]
avg_projects = commit_messages_final.loc[commit_messages_final['project_id'].isin(avglist)]
low_projects = commit_messages_final.loc[commit_messages_final['project_id'].isin(lowlist)]

In [ ]:
#get bin percentages for large projects
large_by_sentiment = large_projects.groupby(['final_sentiment']).count().reset_index()
large_by_sentiment = large_by_sentiment[['final_sentiment','project_id']]
large_by_sentiment.rename(columns={'project_id':'count'}, inplace=True)
large_by_sentiment['percent_of_total'] = large_by_sentiment['count'].divide(len(large_projects)).multiply(100)
large_by_sentiment['bin'] = pd.cut(large_by_sentiment['final_sentiment'], [-5, -1, 0, 4], labels=['negative', 'neutral', 'positive'])
large_by_sentiment_binned = large_by_sentiment.groupby(['bin']).sum().reset_index()
large_by_sentiment_binned = large_by_sentiment_binned[['bin','count','percent_of_total']]
large_by_sentiment_binned

In [ ]:
#get bin percentages for average projects
avg_by_sentiment = avg_projects.groupby(['final_sentiment']).count().reset_index()
avg_by_sentiment = avg_by_sentiment[['final_sentiment','project_id']]
avg_by_sentiment.rename(columns={'project_id':'count'}, inplace=True)
avg_by_sentiment['percent_of_total'] = avg_by_sentiment['count'].divide(len(avg_projects)).multiply(100)
avg_by_sentiment['bin'] = pd.cut(avg_by_sentiment['final_sentiment'], [-5, -1, 0, 4], labels=['negative', 'neutral', 'positive'])
avg_by_sentiment_binned = avg_by_sentiment.groupby(['bin']).sum().reset_index()
avg_by_sentiment_binned = avg_by_sentiment_binned[['bin','count','percent_of_total']]
avg_by_sentiment_binned

In [ ]:
#get bin percentages for low projects
low_by_sentiment = low_projects.groupby(['final_sentiment']).count().reset_index()
low_by_sentiment = low_by_sentiment[['final_sentiment','project_id']]
low_by_sentiment.rename(columns={'project_id':'count'}, inplace=True)
low_by_sentiment['percent_of_total'] = low_by_sentiment['count'].divide(len(low_projects)).multiply(100)
low_by_sentiment['bin'] = pd.cut(low_by_sentiment['final_sentiment'], [-5, -1, 0, 4], labels=['negative', 'neutral', 'positive'])
low_by_sentiment_binned = low_by_sentiment.groupby(['bin']).sum().reset_index()
low_by_sentiment_binned = low_by_sentiment_binned[['bin','count','percent_of_total']]
low_by_sentiment_binned

## RQ2

### Sort out day of week data for all commits

In [ ]:
##create the bins per weekday
weekdays = commit_messages_final.groupby(['day_of_week',pd.cut(commit_messages_final.final_sentiment, bins = [-5, -1, 0, 4], labels=['negative', 'neutral', 'positive'])])
weekdays = weekdays.size().unstack()
weekdays.columns = [''.join(col).strip() for col in weekdays.columns.values]
weekdays.reset_index(inplace=True)

#get the totals again
weekdays['total'] = weekdays['negative'] + weekdays['neutral'] + weekdays['positive']
weekdays['negative_pct'] = weekdays['negative'].divide(weekdays['total']).multiply(100)
weekdays['neutral_pct'] = weekdays['neutral'].divide(weekdays['total']).multiply(100)
weekdays['positive_pct'] = weekdays['positive'].divide(weekdays['total']).multiply(100)

#a rube goldberg way to sort by day of week... FML
days = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday', 'Sunday']
mapping = {day: i for i, day in enumerate(days)}
key = weekdays['day_of_week'].map(mapping)
weekdays = weekdays.iloc[key.argsort()].set_index('day_of_week')
weekdays.columns = [''.join(col).strip() for col in weekdays.columns.values]
weekdays.reset_index(inplace=True)
weekdays

### Sort out day of week data for large commits

In [ ]:
##create the bins per weekday
weekdays_large = large_projects.groupby(['day_of_week',pd.cut(large_projects.final_sentiment, bins = [-5, -1, 0, 4], labels=['negative', 'neutral', 'positive'])])
weekdays_large = weekdays_large.size().unstack()
weekdays_large.columns = [''.join(col).strip() for col in weekdays_large.columns.values]
weekdays_large.reset_index(inplace=True)

#get the totals again
weekdays_large['total'] = weekdays_large['negative'] + weekdays_large['neutral'] + weekdays_large['positive']
weekdays_large['negative_pct'] = weekdays_large['negative'].divide(weekdays_large['total']).multiply(100)
weekdays_large['neutral_pct'] = weekdays_large['neutral'].divide(weekdays_large['total']).multiply(100)
weekdays_large['positive_pct'] = weekdays_large['positive'].divide(weekdays_large['total']).multiply(100)

#a rube goldberg way to sort by day of week... FML
days = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday', 'Sunday']
mapping = {day: i for i, day in enumerate(days)}
key = weekdays_large['day_of_week'].map(mapping)
weekdays_large = weekdays_large.iloc[key.argsort()].set_index('day_of_week')
weekdays_large.columns = [''.join(col).strip() for col in weekdays_large.columns.values]
weekdays_large.reset_index(inplace=True)
weekdays_large

### Sort out day of week data for average commits

In [ ]:
##create the bins per weekday
weekdays_avg = avg_projects.groupby(['day_of_week',pd.cut(avg_projects.final_sentiment, bins = [-5, -1, 0, 4], labels=['negative', 'neutral', 'positive'])])
weekdays_avg = weekdays_avg.size().unstack()
weekdays_avg.columns = [''.join(col).strip() for col in weekdays_avg.columns.values]
weekdays_avg.reset_index(inplace=True)

#get the totals again
weekdays_avg['total'] = weekdays_avg['negative'] + weekdays_avg['neutral'] + weekdays_avg['positive']
weekdays_avg['negative_pct'] = weekdays_avg['negative'].divide(weekdays_avg['total']).multiply(100)
weekdays_avg['neutral_pct'] = weekdays_avg['neutral'].divide(weekdays_avg['total']).multiply(100)
weekdays_avg['positive_pct'] = weekdays_avg['positive'].divide(weekdays_avg['total']).multiply(100)

#a rube goldberg way to sort by day of week... FML
days = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday', 'Sunday']
mapping = {day: i for i, day in enumerate(days)}
key = weekdays_avg['day_of_week'].map(mapping)
weekdays_avg = weekdays_avg.iloc[key.argsort()].set_index('day_of_week')
weekdays_avg.columns = [''.join(col).strip() for col in weekdays_avg.columns.values]
weekdays_avg.reset_index(inplace=True)
weekdays_avg

### Sort out day of week data for low commits

In [ ]:
##create the bins per weekday
weekdays_low = low_projects.groupby(['day_of_week',pd.cut(low_projects.final_sentiment, bins = [-5, -1, 0, 4], labels=['negative', 'neutral', 'positive'])])
weekdays_low = weekdays_low.size().unstack()
weekdays_low.columns = [''.join(col).strip() for col in weekdays_low.columns.values]
weekdays_low.reset_index(inplace=True)

#get the totals again
weekdays_low['total'] = weekdays_low['negative'] + weekdays_low['neutral'] + weekdays_low['positive']
weekdays_low['negative_pct'] = weekdays_low['negative'].divide(weekdays_low['total']).multiply(100)
weekdays_low['neutral_pct'] = weekdays_low['neutral'].divide(weekdays_low['total']).multiply(100)
weekdays_low['positive_pct'] = weekdays_low['positive'].divide(weekdays_low['total']).multiply(100)

#a rube goldberg way to sort by day of week... FML
days = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday', 'Sunday']
mapping = {day: i for i, day in enumerate(days)}
key = weekdays_low['day_of_week'].map(mapping)
weekdays_low = weekdays_low.iloc[key.argsort()].set_index('day_of_week')
weekdays_low.columns = [''.join(col).strip() for col in weekdays_low.columns.values]
weekdays_low.reset_index(inplace=True)
weekdays_low

### graphing

In [ ]:
#graphing the data for all commits
ax = weekdays.plot.bar(x='day_of_week', y='total', color='grey', label='total_commits')
ax2 = weekdays['negative_pct'].plot(secondary_y=True, label='percent of commits that are negative')
ax3 = weekdays['positive_pct'].plot(secondary_y=True, label='percent of commits that are positive')
ax.set_ylabel('num commits')
ax2.set_ylabel('sentiment percentage')
plt.legend(loc='center left', bbox_to_anchor=(1.2, 0.5))
plt.show()

In [ ]:
#graphing the data for large commits
ax = weekdays_large.plot.bar(x='day_of_week', y='total', color='grey', label='total_commits')
ax2 = weekdays_large['negative_pct'].plot(secondary_y=True, label='percent of commits that are negative')
ax3 = weekdays_large['positive_pct'].plot(secondary_y=True, label='percent of commits that are positive')
ax.set_ylabel('num commits')
ax2.set_ylabel('sentiment percentage')
plt.legend(loc='center left', bbox_to_anchor=(1.2, 0.5))
plt.show()
#enterprise programmers dont want to work on the weekend?

In [ ]:
#graphing the data for average commits
ax = weekdays_avg.plot.bar(x='day_of_week', y='total', color='grey', label='total_commits')
ax2 = weekdays_avg['negative_pct'].plot(secondary_y=True, label='percent of commits that are negative')
ax3 = weekdays_avg['positive_pct'].plot(secondary_y=True, label='percent of commits that are positive')
ax.set_ylabel('num commits')
ax2.set_ylabel('sentiment percentage')
plt.legend(loc='center left', bbox_to_anchor=(1.2, 0.5))
plt.show()

In [ ]:
#graphing the data for low commits
ax = weekdays_low.plot.bar(x='day_of_week', y='total', color='grey', label='total_commits')
ax2 = weekdays_low['negative_pct'].plot(secondary_y=True, label='percent of commits that are negative')
ax3 = weekdays_low['positive_pct'].plot(secondary_y=True, label='percent of commits that are positive')
ax.set_ylabel('num commits')
ax2.set_ylabel('sentiment percentage')
plt.legend(loc='center left', bbox_to_anchor=(1.2, 0.5))
plt.show()
#perhaps hobby programmers who are happy to work on their small projects on the weekend

## RQ3
Is there a correlation between the number of changed files and developer sentiment? 

In [ ]:
#count up all the changed files
files_changed_count = files_changed_final.groupby('commit_id').count().reset_index()
files_changed_count = files_changed_count[['commit_id', 'modification']]
files_changed_count.commit_id = files_changed_count.commit_id.astype(str)
files_changed_count.commit_id = files_changed_count.commit_id.str.strip()

In [ ]:
commit_messages_rq3 = commit_messages_final
commit_messages_rq3.dropna(inplace=True)
commit_messages_rq3.commit_id = commit_messages_rq3.commit_id.astype(str)
commit_messages_rq3.commit_id = commit_messages_rq3.commit_id.str.strip()

##### Merge Dataframes:

In [ ]:
commit_file_count = files_changed_count.merge(commit_messages_rq3,on='commit_id', how='left')
commit_file_count = commit_file_count[(commit_file_count.year >= 2008) & (commit_file_count.year <= 2015) ]

In [ ]:
commit_file_count_large = commit_file_count.loc[commit_file_count['project_id'].isin(largelist)]
commit_file_count_avg = commit_file_count.loc[commit_file_count['project_id'].isin(avglist)]
commit_file_count_low = commit_file_count.loc[commit_file_count['project_id'].isin(lowlist)]

In [ ]:
#do some really nasty rearanging of indexes
years_large = commit_file_count_large.groupby(['year',pd.cut(commit_file_count_large.final_sentiment, bins = [-5, -1, 0, 4], labels=['negative', 'neutral', 'positive'])]).sum()
years_large.columns = ['num_files_modified','garbage','garbage2']
years_large = years_large.unstack()
years_large.columns = [' '.join(col).strip() for col in years_large.columns.values]
years_large.reset_index(inplace=True)
years_large = years_large[['year','num_files_modified negative','num_files_modified neutral','num_files_modified positive']]
years_large.columns = ['year','negative_mod_count','neutral_mod_count','positive_mod_count']

temp=commit_file_count_large.groupby(['year']).mean()
temp.reset_index(inplace=True)
temp = temp[['year','modification']]
temp.columns = ['year','avg_files_modified']

years_large = years_large.merge(temp,on='year')
years_large

In [ ]:
years_avg = commit_file_count_avg.groupby(['year',pd.cut(commit_file_count_avg.final_sentiment, bins = [-5, -1, 0, 4], labels=['negative', 'neutral', 'positive'])]).sum()
years_avg.columns = ['num_files_modified','garbage','garbage2']
years_avg = years_avg.unstack()
years_avg.columns = [' '.join(col).strip() for col in years_avg.columns.values]
years_avg.reset_index(inplace=True)
years_avg = years_avg[['year','num_files_modified negative','num_files_modified neutral','num_files_modified positive']]
years_avg.columns = ['year','negative_mod_count','neutral_mod_count','positive_mod_count']

temp=commit_file_count_avg.groupby(['year']).mean()
temp.reset_index(inplace=True)
temp = temp[['year','modification']]
temp.columns = ['year','avg_files_modified']

years_avg = years_avg.merge(temp,on='year')
years_avg

In [ ]:
years_low = commit_file_count_low.groupby(['year',pd.cut(commit_file_count_low.final_sentiment, bins = [-5, -1, 0, 4], labels=['negative', 'neutral', 'positive'])]).sum()
years_low.columns = ['num_files_modified','garbage','garbage2']
years_low = years_low.unstack()
years_low.columns = [' '.join(col).strip() for col in years_low.columns.values]
years_low.reset_index(inplace=True)
years_low = years_low[['year','num_files_modified negative','num_files_modified neutral','num_files_modified positive']]
years_low.columns = ['year','negative_mod_count','neutral_mod_count','positive_mod_count']
years_low = years_low.fillna(0)

temp=commit_file_count_low.groupby(['year']).mean()
temp.reset_index(inplace=True)
temp = temp[['year','modification']]
temp.columns = ['year','avg_files_modified']

years_low = years_low.merge(temp,on='year')
years_low

In [ ]:
#graphing the data for large commits
ax = years_large.plot.bar(x='year', y='avg_files_modified', color='grey', label='avg changed files/commit')
ax2 = years_large['negative_mod_count'].plot(secondary_y=True, label='modifications belonging to negative commit')
ax2 = years_large['neutral_mod_count'].plot(secondary_y=True, label='modifications belonging to neutral commit')
ax3 = years_large['positive_mod_count'].plot(secondary_y=True, label='modifications belonging to positive commit')
ax.set_ylabel('avg changed files/commit')
ax2.set_ylabel('number of files modified')
plt.legend(loc='center left', bbox_to_anchor=(1.2, 0.5))
plt.show()

In [ ]:
#graphing the data for average commits
ax = years_avg.plot.bar(x='year', y='avg_files_modified', color='grey', label='avg changed files/commit')
ax2 = years_avg['negative_mod_count'].plot(secondary_y=True, label='modifications belonging to negative commit')
ax2 = years_avg['neutral_mod_count'].plot(secondary_y=True, label='modifications belonging to neutral commit')
ax3 = years_avg['positive_mod_count'].plot(secondary_y=True, label='modifications belonging to positive commit')
ax.set_ylabel('avg changed files/commit')
ax2.set_ylabel('number of files modified')
plt.legend(loc='center left', bbox_to_anchor=(1.2, 0.5))
plt.show()

In [ ]:
#graphing the data for low commits
ax = years_low.plot.bar(x='year', y='avg_files_modified', color='grey', label='avg changed files/commit')
ax2 = years_low['negative_mod_count'].plot(secondary_y=True, label='modifications belonging to negative commit')
ax2 = years_low['neutral_mod_count'].plot(secondary_y=True, label='modifications belonging to neutral commit')
ax3 = years_low['positive_mod_count'].plot(secondary_y=True, label='modifications belonging to positive commit')
ax.set_ylabel('avg changed files/commit')
ax2.set_ylabel('number of files modified')
plt.legend(loc='center left', bbox_to_anchor=(1.2, 0.5))
plt.show()

# RQ4

Is there a correlation between type of file modification and sentiment

In [ ]:
#cound the number of each type of modification for each commit
files_changed_rq4 = files_changed_final.groupby(['modification','commit_id']).count().reset_index()
files_changed_rq4 = files_changed_rq4.pivot(index='commit_id', columns='modification', values='filepath')
files_changed_rq4.columns = [''.join(col).strip() for col in files_changed_rq4.columns.values]
files_changed_rq4.reset_index(inplace=True)
files_changed_rq4.fillna(0, inplace=True)

In [ ]:
#get the totals again and get percentages
files_changed_rq4['total'] = files_changed_rq4['Add'] + files_changed_rq4['Delete'] + files_changed_rq4['Modify']
files_changed_rq4['add_pct'] = files_changed_rq4['Add'].divide(files_changed_rq4['total']).multiply(100)
files_changed_rq4['delete_pct'] = files_changed_rq4['Delete'].divide(files_changed_rq4['total']).multiply(100)
files_changed_rq4['modify_pct'] = files_changed_rq4['Modify'].divide(files_changed_rq4['total']).multiply(100)

In [ ]:
#get commit messages and strip any nasty hidden characters
commit_messages_rq4 = commit_messages_final
commit_messages_rq4.dropna(inplace=True)
commit_messages_rq4.commit_id = commit_messages_rq4.commit_id.astype(str)
commit_messages_rq4.commit_id = commit_messages_rq4.commit_id.str.strip()

files_changed_rq4.commit_id = files_changed_rq4.commit_id.astype(str)
files_changed_rq4.commit_id = files_changed_rq4.commit_id.str.strip()

In [ ]:
#break apart the three size groups
commit_mod_count = files_changed_rq4.merge(commit_messages_rq4,on='commit_id', how='left')
commit_mod_count_large = commit_mod_count.loc[commit_mod_count['project_id'].isin(largelist)]
commit_mod_count_avg = commit_mod_count.loc[commit_mod_count['project_id'].isin(avglist)]
commit_mod_count_low = commit_mod_count.loc[commit_mod_count['project_id'].isin(lowlist)]

In [ ]:
#group by whichever type of modification is greater than 80% of the modifications in each commit
#throw away commits that arent strongly leaning toward one type, as in not greater than 80% adds, modifies, or deletes
commit_mod_count_large.loc[(commit_mod_count_large.add_pct >= 80), 'majority_mod'] = 'add'
commit_mod_count_large.loc[(commit_mod_count_large.delete_pct >= 80), 'majority_mod'] = 'delete' 
commit_mod_count_large.loc[(commit_mod_count_large.modify_pct >= 80), 'majority_mod'] = 'modify' 
commit_mod_count_large.dropna(inplace=True)
commit_mod_count_large = commit_mod_count_large.groupby(['majority_mod',pd.cut(commit_mod_count_large.final_sentiment, bins = [-5, -1, 0, 4], labels=['negative', 'neutral', 'positive'])]).count()
commit_mod_count_large = commit_mod_count_large.unstack()
commit_mod_count_large.columns = [' '.join(col).strip() for col in commit_mod_count_large.columns.values]
commit_mod_count_large.reset_index(inplace=True)
commit_mod_count_large = commit_mod_count_large[['majority_mod','commit_id negative','commit_id neutral','commit_id positive']]
commit_mod_count_large.columns = ['majority_mod','negative_commit_count','neutral_commit_count','positive_commit_count']

#get the totals again
commit_mod_count_large['total'] = commit_mod_count_large['negative_commit_count'] + commit_mod_count_large['neutral_commit_count'] + commit_mod_count_large['positive_commit_count']
commit_mod_count_large['negative_pct'] = commit_mod_count_large['negative_commit_count'].divide(commit_mod_count_large['total']).multiply(100)
commit_mod_count_large['neutral_pct'] = commit_mod_count_large['neutral_commit_count'].divide(commit_mod_count_large['total']).multiply(100)
commit_mod_count_large['positive_pct'] = commit_mod_count_large['positive_commit_count'].divide(commit_mod_count_large['total']).multiply(100)

commit_mod_count_large

In [ ]:
#group by whichever type of modification is greater than 80% of the modifications in each commit
#throw away commits that arent strongly leaning toward one type, as in not greater than 80% adds, modifies, or deletes
commit_mod_count_avg.loc[(commit_mod_count_avg.add_pct >= 80), 'majority_mod'] = 'add'
commit_mod_count_avg.loc[(commit_mod_count_avg.delete_pct >= 80), 'majority_mod'] = 'delete' 
commit_mod_count_avg.loc[(commit_mod_count_avg.modify_pct >= 80), 'majority_mod'] = 'modify' 
commit_mod_count_avg.dropna(inplace=True)
commit_mod_count_avg = commit_mod_count_avg.groupby(['majority_mod',pd.cut(commit_mod_count_avg.final_sentiment, bins = [-5, -1, 0, 4], labels=['negative', 'neutral', 'positive'])]).count()
commit_mod_count_avg = commit_mod_count_avg.unstack()
commit_mod_count_avg.columns = [' '.join(col).strip() for col in commit_mod_count_avg.columns.values]
commit_mod_count_avg.reset_index(inplace=True)
commit_mod_count_avg = commit_mod_count_avg[['majority_mod','commit_id negative','commit_id neutral','commit_id positive']]
commit_mod_count_avg.columns = ['majority_mod','negative_commit_count','neutral_commit_count','positive_commit_count']

#get the totals again
commit_mod_count_avg['total'] = commit_mod_count_avg['negative_commit_count'] + commit_mod_count_avg['neutral_commit_count'] + commit_mod_count_avg['positive_commit_count']
commit_mod_count_avg['negative_pct'] = commit_mod_count_avg['negative_commit_count'].divide(commit_mod_count_avg['total']).multiply(100)
commit_mod_count_avg['neutral_pct'] = commit_mod_count_avg['neutral_commit_count'].divide(commit_mod_count_avg['total']).multiply(100)
commit_mod_count_avg['positive_pct'] = commit_mod_count_avg['positive_commit_count'].divide(commit_mod_count_avg['total']).multiply(100)

commit_mod_count_avg

In [ ]:
#group by whichever type of modification is greater than 80% of the modifications in each commit
#throw away commits that arent strongly leaning toward one type, as in not greater than 80% adds, modifies, or deletes
commit_mod_count_low.loc[(commit_mod_count_low.add_pct >= 80), 'majority_mod'] = 'add'
commit_mod_count_low.loc[(commit_mod_count_low.delete_pct >= 80), 'majority_mod'] = 'delete' 
commit_mod_count_low.loc[(commit_mod_count_low.modify_pct >= 80), 'majority_mod'] = 'modify' 
commit_mod_count_low.dropna(inplace=True)
commit_mod_count_low = commit_mod_count_low.groupby(['majority_mod',pd.cut(commit_mod_count_low.final_sentiment, bins = [-5, -1, 0, 4], labels=['negative', 'neutral', 'positive'])]).count()
commit_mod_count_low = commit_mod_count_low.unstack()
commit_mod_count_low.columns = [' '.join(col).strip() for col in commit_mod_count_low.columns.values]
commit_mod_count_low.reset_index(inplace=True)
commit_mod_count_low = commit_mod_count_low[['majority_mod','commit_id negative','commit_id neutral','commit_id positive']]
commit_mod_count_low.columns = ['majority_mod','negative_commit_count','neutral_commit_count','positive_commit_count']
commit_mod_count_low.fillna(0, inplace=True)

#get the totals again
commit_mod_count_low['total'] = commit_mod_count_low['negative_commit_count'] + commit_mod_count_low['neutral_commit_count'] + commit_mod_count_low['positive_commit_count']
commit_mod_count_low['negative_pct'] = commit_mod_count_low['negative_commit_count'].divide(commit_mod_count_low['total']).multiply(100)
commit_mod_count_low['neutral_pct'] = commit_mod_count_low['neutral_commit_count'].divide(commit_mod_count_low['total']).multiply(100)
commit_mod_count_low['positive_pct'] = commit_mod_count_low['positive_commit_count'].divide(commit_mod_count_low['total']).multiply(100)

commit_mod_count_low

In [ ]:
#graphing the data for large projects
ax = commit_mod_count_large.plot.bar(x='majority_mod', y='total', color='grey', label='total_commits')
ax1 = commit_mod_count_large['neutral_pct'].plot(secondary_y=True, label='percent of commits that are neutral')
ax2 = commit_mod_count_large['negative_pct'].plot(secondary_y=True, label='percent of commits that are negative')
ax3 = commit_mod_count_large['positive_pct'].plot(secondary_y=True, label='percent of commits that are positive')
ax.set_ylabel('num commits')
ax2.set_ylabel('sentiment percentage')
plt.legend(loc='center left', bbox_to_anchor=(1.2, 0.5))
plt.show()

In [ ]:
#graphing the data for average projects
ax = commit_mod_count_avg.plot.bar(x='majority_mod', y='total', color='grey', label='total_commits')
ax1 = commit_mod_count_avg['neutral_pct'].plot(secondary_y=True, label='percent of commits that are neutral')
ax2 = commit_mod_count_avg['negative_pct'].plot(secondary_y=True, label='percent of commits that are negative')
ax3 = commit_mod_count_avg['positive_pct'].plot(secondary_y=True, label='percent of commits that are positive')
ax.set_ylabel('num commits')
ax2.set_ylabel('sentiment percentage')
plt.legend(loc='center left', bbox_to_anchor=(1.2, 0.5))
plt.show()

In [ ]:
#graphing the data for low projects
ax = commit_mod_count_low.plot.bar(x='majority_mod', y='total', color='grey', label='total_commits')
ax1 = commit_mod_count_low['neutral_pct'].plot(secondary_y=True, label='percent of commits that are neutral')
ax2 = commit_mod_count_low['negative_pct'].plot(secondary_y=True, label='percent of commits that are negative')
ax3 = commit_mod_count_low['positive_pct'].plot(secondary_y=True, label='percent of commits that are positive')
ax.set_ylabel('num commits')
ax2.set_ylabel('sentiment percentage')
plt.legend(loc='center left', bbox_to_anchor=(1.2, 0.5))
plt.show()

# RQ5